In [ ]:
# Virtual Mouse using Hand Gestures
# Required installations:
# !pip install opencv-python
# !pip install mediapipe
# !pip install pyautogui
# !pip install numpy

import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time

# Initialize MediaPipe Hand solutions
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Get screen dimensions
screen_width, screen_height = pyautogui.size()

rtsp_url = "rtsp://192.168.18.38:554"

# Initialize camera
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(rtsp_url)



# Variables for mouse control
prev_x, prev_y = 0, 0
smooth_factor = 5  # Higher for smoother movement
click_delay = 0.3  # Time delay between clicks to prevent multiple clicks
last_click_time = time.time()
clicking_mode = False
frame_counter = 0
scrolling_mode = False

# Parameters for gesture control
click_distance_threshold = 0.04  # Distance between index and thumb for click
scroll_threshold = 0.1  # Distance threshold for scroll detection

def calculate_distance(p1, p2):
    """Calculate Euclidean distance between two points"""
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Failed to capture image")
            continue
        
        # Flip the image horizontally for a more intuitive mirror view
        image = cv2.flip(image, 1)
        
        # Convert the image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the image and detect hands
        results = hands.process(image_rgb)
        
        # Draw hand landmarks on the image
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Extract index finger tip coordinates
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                index_x = int(index_finger_tip.x * image.shape[1])
                index_y = int(index_finger_tip.y * image.shape[0])
                
                # Extract thumb tip coordinates
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                thumb_x = int(thumb_tip.x * image.shape[1])
                thumb_y = int(thumb_tip.y * image.shape[0])
                
                # Extract middle finger tip coordinates for additional gestures
                middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                middle_x = int(middle_finger_tip.x * image.shape[1])
                middle_y = int(middle_finger_tip.y * image.shape[0])
                
                # Map coordinates to screen
                screen_x = np.interp(index_x, [100, image.shape[1]-100], [0, screen_width])
                screen_y = np.interp(index_y, [100, image.shape[0]-100], [0, screen_height])
                
                # Smooth the movement
                current_x = prev_x + (screen_x - prev_x) / smooth_factor
                current_y = prev_y + (screen_y - prev_y) / smooth_factor
                
                # Move the mouse
                pyautogui.moveTo(current_x, current_y)
                prev_x, prev_y = current_x, current_y
                
                # Draw a circle at index finger position
                cv2.circle(image, (index_x, index_y), 10, (0, 255, 0), -1)
                
                # Calculate distance between index finger and thumb for click detection
                distance = calculate_distance(
                    (index_finger_tip.x, index_finger_tip.y),
                    (thumb_tip.x, thumb_tip.y)
                )
                
                # Check for click gesture (thumb and index finger pinch)
                if distance < click_distance_threshold:
                    # Add visual feedback
                    cv2.circle(image, (thumb_x, thumb_y), 10, (0, 0, 255), -1)
                    
                    # Click with delay to prevent multiple clicks
                    current_time = time.time()
                    if current_time - last_click_time > click_delay:
                        pyautogui.click()
                        last_click_time = current_time
                        cv2.putText(image, "Click!", (50, 50), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                # Check for scroll gesture (middle finger and thumb distance)
                middle_thumb_distance = calculate_distance(
                    (middle_finger_tip.x, middle_finger_tip.y),
                    (thumb_tip.x, thumb_tip.y)
                )
                
                if middle_thumb_distance < click_distance_threshold:
                    scrolling_mode = True
                    # Use the vertical movement of the hand to scroll
                    if frame_counter % 5 == 0:  # Reduce scroll sensitivity
                        if middle_y < image.shape[0] // 2:
                            pyautogui.scroll(10)  # Scroll up
                            cv2.putText(image, "Scroll Up", (50, 100), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        else:
                            pyautogui.scroll(-10)  # Scroll down
                            cv2.putText(image, "Scroll Down", (50, 100), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                else:
                    scrolling_mode = False
                
                # Right-click gesture (ring finger and thumb pinch)
                ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                ring_thumb_distance = calculate_distance(
                    (ring_finger_tip.x, ring_finger_tip.y),
                    (thumb_tip.x, thumb_tip.y)
                )
                
                if ring_thumb_distance < click_distance_threshold:
                    current_time = time.time()
                    if current_time - last_click_time > click_delay:
                        pyautogui.rightClick()
                        last_click_time = current_time
                        cv2.putText(image, "Right Click!", (50, 150), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        
        # Display control instructions
        cv2.putText(image, "Index + Thumb Pinch: Left Click", (10, image.shape[0] - 120), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "Middle + Thumb Pinch: Scroll", (10, image.shape[0] - 90), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "Ring + Thumb Pinch: Right Click", (10, image.shape[0] - 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "Press 'q' to quit", (10, image.shape[0] - 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Show the image
        cv2.imshow('Virtual Mouse Controller', image)
        
        # Increment frame counter
        frame_counter += 1
        
        # Exit if 'q' is pressed
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()